# **Install libararies**

In [ ]:
pip install -U peft
!pip install -U autotrain-advanced -q
!pip install diffusers --upgrade
!pip install invisible_watermark transformers accelerate safetensors
pip install insightface
pip install codeformer-pip
pip install numpy==1.26.4

# **Necessary Importing Libararies**

In [1]:
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt
#import insightface
from insightface.app import FaceAnalysis # face detetction model
from insightface.data import get_image # detect face
from PIL import Image


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# **Hugging Face Auth**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


# **Text 2 image model loading**

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch
import peftv  

prj_path = "fahd9999/my-dreambooth-project"
model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
     model,
     torch_dtype=torch.float16,use_safetensors=True, variant="fp16"
 )
pipe.to("cuda")
pipe.load_lora_weights(prj_path, weight_name="pytorch_lora_weights.safetensors")

# ***Loading Face detection model***

In [ ]:
import cv2
from insightface.app import FaceAnalysis

# Initialize the FaceAnalysis application
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(640, 640))

def check_face_in_image(image_path):
    img = cv2.imread(image_path)
    faces = app.get(img)
    print(faces)
    # Set the flag based on whether a face is detected
    face_detected_flag = len(faces) > 0
    
    return face_detected_flag

image_path = r'2.webp' # add your own image path 
is_face_detected = check_face_in_image(image_path)
print(f"Face detected: {is_face_detected}")


# ***Loading face swaping model***

1.   Model is on my drive
2.   Loading model using insight face and saving to face_swap_obj



In [ ]:
face_swap_obj = insightface.model_zoo.get_model(r'/content/drive/My Drive/inswapper_128.onnx', download=False, download_zip=False)

# **Face Swaping Main**

*   Takes face_swapper(model)
*   original_image is target image
*   Source image is users face
*   Source face and target face is extracted through face detection model


In [9]:
def face_swaping(face_swapper,orginal_image,source_face,target_faces):

    result_image = orginal_image.copy()
    for i, face in enumerate(target_faces):
        result_image = face_swapper.get(result_image, face, source_face, paste_back=True)
    return result_image

In [11]:
def image_generation(source_image,target_image):
  global face_swap_obj #inswapper model object
  target_faces=app.get(img=target_image)
  source_face=app.get(img=source_image)
  source_face=source_face[0]
  face_swapped_image=face_swaping(face_swap_obj,target_image,source_face,target_faces)
  plt.imshow(face_swapped_image)
  return face_swapped_image

# **Image Enhancer Upscaler Model**

In [12]:
import os

import cv2
import torch
from torchvision.transforms.functional import normalize

from codeformer.basicsr.archs.rrdbnet_arch import RRDBNet
from codeformer.basicsr.utils import img2tensor, imwrite, tensor2img
from codeformer.basicsr.utils.download_util import load_file_from_url
from codeformer.basicsr.utils.realesrgan_utils import RealESRGANer
from codeformer.basicsr.utils.registry import ARCH_REGISTRY
from codeformer.facelib.utils.face_restoration_helper import FaceRestoreHelper
from codeformer.facelib.utils.misc import is_gray

pretrain_model_url = {
    "codeformer": "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth",
    "detection": "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/detection_Resnet50_Final.pth",
    "parsing": "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/parsing_parsenet.pth",
    "realesrgan": "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/RealESRGAN_x2plus.pth",
}

# download weights
if not os.path.exists("CodeFormer/weights/CodeFormer/codeformer.pth"):
    load_file_from_url(
        url=pretrain_model_url["codeformer"], model_dir="CodeFormer/weights/CodeFormer", progress=True, file_name=None
    )
if not os.path.exists("CodeFormer/weights/facelib/detection_Resnet50_Final.pth"):
    load_file_from_url(
        url=pretrain_model_url["detection"], model_dir="CodeFormer/weights/facelib", progress=True, file_name=None
    )
if not os.path.exists("CodeFormer/weights/facelib/parsing_parsenet.pth"):
    load_file_from_url(
        url=pretrain_model_url["parsing"], model_dir="CodeFormer/weights/facelib", progress=True, file_name=None
    )
if not os.path.exists("CodeFormer/weights/realesrgan/RealESRGAN_x2plus.pth"):
    load_file_from_url(
        url=pretrain_model_url["realesrgan"], model_dir="CodeFormer/weights/realesrgan", progress=True, file_name=None
    )


def imread(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


# set enhancer with RealESRGAN
def set_realesrgan():
    half = True if torch.cuda.is_available() else False
    model = RRDBNet(
        num_in_ch=3,
        num_out_ch=3,
        num_feat=64,
        num_block=23,
        num_grow_ch=32,
        scale=2,
    )
    upsampler = RealESRGANer(
        scale=2,
        model_path="CodeFormer/weights/realesrgan/RealESRGAN_x2plus.pth",
        model=model,
        tile=400,
        tile_pad=40,
        pre_pad=0,
        half=half,
    )
    return upsampler


upsampler = set_realesrgan()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
codeformer_net = ARCH_REGISTRY.get("CodeFormer")(
    dim_embd=512,
    codebook_size=1024,
    n_head=8,
    n_layers=9,
    connect_list=["32", "64", "128", "256"],
).to(device)
ckpt_path = "CodeFormer/weights/CodeFormer/codeformer.pth"
checkpoint = torch.load(ckpt_path)["params_ema"]
codeformer_net.load_state_dict(checkpoint)
codeformer_net.eval()

os.makedirs("output", exist_ok=True)


def inference_app(image, background_enhance, face_upsample, upscale, codeformer_fidelity,file_name):
    # take the default setting for the demo
    has_aligned = False
    only_center_face = False
    draw_box = False
    detection_model = "retinaface_resnet50"
    print("Inp:", image, background_enhance, face_upsample, upscale, codeformer_fidelity)

    img = cv2.imread(str(image), cv2.IMREAD_COLOR)
    print("\timage size:", img.shape)

    upscale = int(upscale)  # convert type to int
    if upscale > 4:  # avoid memory exceeded due to too large upscale
        upscale = 4
    if upscale > 2 and max(img.shape[:2]) > 1000:  # avoid memory exceeded due to too large img resolution
        upscale = 2
    if max(img.shape[:2]) > 1500:  # avoid memory exceeded due to too large img resolution
        upscale = 1
        background_enhance = False
        face_upsample = False

    face_helper = FaceRestoreHelper(
        upscale,
        face_size=512,
        crop_ratio=(1, 1),
        det_model=detection_model,
        save_ext="png",
        use_parse=True,
        device=device,
    )
    bg_upsampler = upsampler if background_enhance else None
    face_upsampler = upsampler if face_upsample else None

    if has_aligned:
        # the input faces are already cropped and aligned
        img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_LINEAR)
        face_helper.is_gray = is_gray(img, threshold=5)
        if face_helper.is_gray:
            print("\tgrayscale input: True")
        face_helper.cropped_faces = [img]
    else:
        face_helper.read_image(img)
        # get face landmarks for each face
        num_det_faces = face_helper.get_face_landmarks_5(
            only_center_face=only_center_face, resize=640, eye_dist_threshold=5
        )
        print(f"\tdetect {num_det_faces} faces")
        # align and warp each face
        face_helper.align_warp_face()

    # face restoration for each cropped face
    for idx, cropped_face in enumerate(face_helper.cropped_faces):
        # prepare data
        cropped_face_t = img2tensor(cropped_face / 255.0, bgr2rgb=True, float32=True)
        normalize(cropped_face_t, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
        cropped_face_t = cropped_face_t.unsqueeze(0).to(device)

        try:
            with torch.no_grad():
                output = codeformer_net(cropped_face_t, w=codeformer_fidelity, adain=True)[0]
                restored_face = tensor2img(output, rgb2bgr=True, min_max=(-1, 1))
            del output
            torch.cuda.empty_cache()
        except RuntimeError as error:
            print(f"Failed inference for CodeFormer: {error}")
            restored_face = tensor2img(cropped_face_t, rgb2bgr=True, min_max=(-1, 1))

        restored_face = restored_face.astype("uint8")
        face_helper.add_restored_face(restored_face)

    # paste_back
    if not has_aligned:
        # upsample the background
        if bg_upsampler is not None:
            # Now only support RealESRGAN for upsampling background
            bg_img = bg_upsampler.enhance(img, outscale=upscale)[0]
        else:
            bg_img = None
        face_helper.get_inverse_affine(None)
        # paste each restored face to the input image
        if face_upsample and face_upsampler is not None:
            restored_img = face_helper.paste_faces_to_input_image(
                upsample_img=bg_img,
                draw_box=draw_box,
                face_upsampler=face_upsampler,
            )
        else:
            restored_img = face_helper.paste_faces_to_input_image(upsample_img=bg_img, draw_box=draw_box)

    # save restored img
    save_path = f"output/{file_name}.png"
    imwrite(restored_img, str(save_path))
    return save_path


# ***Text to image***
> Takes prompt and generate image and save to img file


In [13]:
def generate_image_from_prompt_and_image(prompt, init_image):
    # Convert the initial image to the appropriate format
    if init_image:
      #init_image = init_image.convert("RGB")

      # Generate an image using the pipeline with reduced inference steps and adjusted dimensions
      #generated_image = pipe(prompt=prompt, image=init_image, strength=0.75, guidance_scale=7.5).images[0]
      generated_image = pipe(prompt,image=init_image, num_inference_steps=50, height=1024, width=1024).images[0]
    else:
      generated_image = pipe(prompt=prompt, num_inference_steps=50, height=1024, width=1024).images[0]


    # Save the image to a bytes buffer
    image_bytes = io.BytesIO()
    generated_image.save(image_bytes, format='PNG')
    image_bytes.seek(0)
    # Save the byte stream to a file
    file_path='genrated_image_text_2_img.png'
    with open(file_path, 'wb') as f:
        f.write(image_bytes.getvalue())

    return file_path

# **Img to bytes**

> Reads image using Pillow
> Then Convert image array to bytes
> Returns bytes object

In [14]:
def image_file_to_bytes(file_path):
    # Open the image file
    with Image.open(file_path) as img:
        # Save the image to a bytes buffer
        image_bytes = io.BytesIO()
        img.save(image_bytes, format='PNG')
        image_bytes.seek(0)  # Move the cursor to the start of the buffer
        return image_bytes#.getvalue()

# ***ProfileGenz Bot***


> Command usage /gen

* User enter /gen >> Upload Source & Target Image  
* Bot Process Img >> Send to Face detection
* Faces BBOX (boundaries will be captured and return)
* Sent to inswapper model for swapping >> return swapped image
* Enhace img and upscale using codeformer RealESRGAN model
* Generate Final Image and Return to user

> Command usage /text2img
* User enter text for image generation
* bot send the prompt to image generation function
* Using SDXL Image will be generated in 42 seconds on current setting of model
* Image will be upscaled and sent to user










In [ ]:
import telebot
from telebot import types, apihelper
from PIL import Image
import io

# First connecting our bot with telegram servers
TOKEN = '7171360134:AAGaHXOrlrzv8r17BTImOybAMIi7dl_V2kM'

# Create a telebot instance with custom request session
bot = telebot.TeleBot(TOKEN)

# Function to handle the /gen command
@bot.message_handler(commands=['gen'])
def start_generation(message):
    bot.reply_to(message, 'Give your face image')
    bot.register_next_step_handler(message, receive_face_image)

# Function to handle receiving the face image
def receive_face_image(message):
    print(message.content_type)
    if message.content_type == 'photo' or message.content_type=="document":
        try:
          file_info = bot.get_file(message.photo[-1].file_id)
        except:
          file_info = bot.get_file(message.document.file_id)
        downloaded_file = bot.download_file(file_info.file_path)

        # Save the image and open with Pillow

        face_image = Image.open(io.BytesIO(downloaded_file))
        face_image.save('face_image.png')

        bot.reply_to(message, 'Face image received. Now send the target image.')
        bot.register_next_step_handler(message, receive_target_image, face_image)
    else:
        bot.reply_to(message, 'Please send an image file.')
        bot.register_next_step_handler(message, receive_face_image)

# Function to handle receiving the target image
def receive_target_image(message, face_image):
    print(f'{message.content_type}')
    if message.content_type == 'photo':
        file_info = bot.get_file(message.photo[-1].file_id)
        downloaded_file = bot.download_file(file_info.file_path)

        # Save the image and open with Pillow
        target_image = Image.open(io.BytesIO(downloaded_file))
        bot.reply_to(message, 'Target image received.\nPlease wait  while we proccess the image it may take few seconds.')

        target_image.save('target_image.png')


        bot.reply_to(message, 'Enhancing your images .')

        # Add your image processing function here
        result_image = image_generation(np.array(face_image), np.array(target_image))
        # Convert the result back to a Pillow image
        result_image = Image.fromarray(result_image)
        result_image_bytes = io.BytesIO()
        result_image.save(result_image_bytes, format='PNG')
        #result_image_bytes.seek(0)
        file_path='genrated_profile_image.png'
        with open(file_path, 'wb') as f:
            f.write(result_image_bytes.getvalue())
        bot.reply_to(message, 'Profile generated successfully Now enhancing and upscaling.')
        final_image=inference_app(
  image=file_path,
  background_enhance=True,
  face_upsample=True,
  upscale=2,
  codeformer_fidelity=0.5,  file_name='generated_image'

)
        #final_image_bytes=image_file_to_bytes(final_image)
        profile_image_bytes=image_file_to_bytes(final_image)
        bot.send_photo(message.chat.id, profile_image_bytes, 'Here is the final image')
    else:
        bot.reply_to(message, 'Please send an image file.')
        bot.register_next_step_handler(message, receive_target_image, face_image)
# Function to handle the /genimage command
@bot.message_handler(commands=['text2img'])
def ask_for_prompt(message):
    bot.reply_to(message, 'Please enter the text for image generation:')
    bot.register_next_step_handler(message, receive_initial_image)

def receive_initial_image(message):
    print(message.content_type)
    if message:  # .content_type == 'photo':
        # Download the photo
        if message.photo:
            file_info = bot.get_file(message.photo[-1].file_id)
            downloaded_file = bot.download_file(file_info.file_path)

            # Ensure the downloaded file is not empty or corrupted
            if not downloaded_file:
                bot.reply_to(message, 'Failed to download the image. Please try again.')
                return

            try:
                # Log the length of the downloaded file for debugging
                print(f"Downloaded file length: {len(downloaded_file)}")

                # Save the downloaded file to disk for further inspection
                with open("downloaded_image.png", "wb") as f:
                    f.write(downloaded_file)

                init_image = Image.open("downloaded_image.png")
                init_image = init_image.convert("RGB")
            except Exception as e:
                print(f"Error opening image: {e}")
                bot.reply_to(message, 'Failed to process the image. Please send a valid image.')
                return
        else:
            init_image = ''

        # text_prompt = message.text  # 'change the background and make image realistic'  # Customize this as needed
        text_prompt = message.caption if message.caption else message.text
        print(f'Received caption: {text_prompt}')
        bot.reply_to(message, f'Text and image received. Generating the image...')
        print(f'Prompt for image generation: {text_prompt}')

        # Generate image from prompt and initial image
        generated_image = generate_image_from_prompt_and_image(text_prompt, init_image)
        final_image=inference_app(
  image=generated_image,
  background_enhance=True,
  face_upsample=True,
  upscale=4,
  codeformer_fidelity=0.5,  file_name='generated_image'

)
        final_image_bytes=image_file_to_bytes(final_image)
        # Send the generated image to the user
        bot.send_photo(message.chat.id, final_image_bytes, 'Here is the generated image')
    else:
        bot.reply_to(message, 'Please send a valid image.')
        bot.register_next_step_handler(message, receive_initial_image)

# Function to handle unrecognized commands
@bot.message_handler(func=lambda m: True)
def echo_all(message):
    bot.reply_to(message, 'Command not recognized. Use /help to see all commands \n')

if __name__ == '__main__':
    bot.polling(none_stop=True)


### Base Prompt

In [15]:
"""
A high-quality, professional LinkedIn profile picture of a businessman. The businessman is wearing a well-tailored dark business suit, a light-colored dress shirt, and a coordinating tie. He has a confident and friendly expression, with a neat haircut and a well-groomed appearance. The background is simple and blurred to ensure the focus remains on the subject. The lighting is even and flattering, highlighting the man's features and creating a polished and professional look. This image should convey professionalism, approachability, and competence, ideal for a LinkedIn profile.
"""